In [ ]:
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

In [1]:
%pip install pygame

Note: you may need to restart the kernel to use updated packages.


Indepndencies

In [1]:
import gymnasium as gym
from stable_baselines3 import A2C
from stable_baselines3.common.vec_env import VecFrameStack
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.env_util import make_atari_env
import pygame
import os


In [5]:
!python -m atari_py.import_roms .\ROMS

e:\Reinforcment-Learning\AtariRLModel\.conda\python.exe: Error while finding module specification for 'atari_py.import_roms' (ModuleNotFoundError: No module named 'atari_py')


Test

In [2]:
environment_name='Breakout-v4'
env=gym.make(environment_name,render_mode='human') 

In [3]:
env.reset()

(array([[[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        ...,
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]]], dtype=uint8),
 {'lives': 5, 'episode_frame_number': 0, 'frame_number': 0})

In [4]:
env.action_space

Discrete(4)

In [5]:
env.observation_space

Box(0, 255, (210, 160, 3), uint8)

In [6]:
episodes=5
for episode in range (1,episodes+1):
    obs=env.reset()
    done=False
    score=0
    while not done:
        env.render()
        action=env.action_space.sample()
        obs, reward, done, any, info = env.step(action)
        score += reward
    print('Episode:{} Score{}'.format(episode,score))
pygame.quit()
env.close()        

e:\Reinforcment-Learning\AtariRLModel\.conda\Lib\site-packages\gymnasium\utils\passive_env_checker.py:335: UserWarning: WARN: No render fps was declared in the environment (env.metadata['render_fps'] is None or not defined), rendering may occur at inconsistent fps.
  logger.warn(


Episode:1 Score1.0
Episode:2 Score2.0
Episode:3 Score0.0
Episode:4 Score1.0
Episode:5 Score1.0
